In [16]:
%matplotlib inline
import random
import torch

batch_size = 20
epoch = 3

def generate_new_data(w,b,number_examples):
  x=torch.normal(mean=0,std=1, size=(number_examples,len(w)))
  y=torch.matmul(x,w)+b
  d=torch.normal(0,0.01,size=(y.shape))

  return x, (y+d).reshape(-1,1)

true_w=torch.tensor([2,-3.4])
true_b=4.2
x, y=generate_new_data(true_w,true_b,100) # generate 100 data

# initialize parameters
w = torch.normal(0,0.01,(2,1),requires_grad=True)
b = torch.zeros(1,requires_grad=True)

# split into mini batch
def data_iter(x, y, batch_size, total_number):
    indices = list(range(total_number))
    random.shuffle(indices)
    for i in range(0,total_number,batch_size):
        batch_indices = torch.tensor(
            indices[i: min(i+batch_size,total_number)]
        )
        yield x[batch_indices], y[batch_indices]
        
# model
def linear(w,b,x):
    return torch.matmul(x,w)+b

# loss fcn
def square_loss(y, y_hat):
    return (y-y_hat.reshape(y.shape)) ** 2 / 2

# parameters update using sgd
def sgd(params, lr):
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad
            param.grad.zero_()
# train
def linear_train(x, y, epochs, batch_size, w, b):
    for epoch in range(epochs):
        for x_mini, y_mini in data_iter(x, y, batch_size, 100):
            y_hat = linear(w,b,x_mini)
            l = square_loss(y_mini, y_hat)
            l.sum().backward()
            sgd([w,b],0.02)
        with torch.no_grad():
            train_loss = square_loss(y, linear(w,b,x))
            print(f'epochs: {epoch}, loss: {float(train_loss.mean()):f}')

linear_train(x, y, epoch, batch_size, w, b)


epochs: 0, loss: 0.118243
epochs: 1, loss: 0.001927
epochs: 2, loss: 0.000089
